In [1]:
import json
import os




In [2]:
from pycocotools.coco import COCO

# 기존 train json path 입력!

In [3]:
!pwd

/data/ephemeral/home/level2-objectdetection-cv-01/mmdetection


In [4]:
# 2. 이미지 메타데이터가 포함된 COCO 형식의 파일 불러오기
image_info_json_file_path = '../../dataset/test.json'
with open(image_info_json_file_path, 'r') as f:
    image_info_data = json.load(f)

# 'images' 정보 추출 (이미지 파일에 대한 메타데이터)
images_info = image_info_data['images']


In [5]:
# COCO 주석 파일을 위한 기본 구조
coco_format = {
    "images": [],
    "annotations": [],
    "categories": [
    {
      "id": 0,
      "name": "General trash",
      "supercategory": "General trash"
    },
    {
      "id": 1,
      "name": "Paper",
      "supercategory": "Paper"
    },
    {
      "id": 2,
      "name": "Paper pack",
      "supercategory": "Paper pack"
    },
    {
      "id": 3,
      "name": "Metal",
      "supercategory": "Metal"
    },
    {
      "id": 4,
      "name": "Glass",
      "supercategory": "Glass"
    },
    {
      "id": 5,
      "name": "Plastic",
      "supercategory": "Plastic"
    },
    {
      "id": 6,
      "name": "Styrofoam",
      "supercategory": "Styrofoam"
    },
    {
      "id": 7,
      "name": "Plastic bag",
      "supercategory": "Plastic bag"
    },
    {
      "id": 8,
      "name": "Battery",
      "supercategory": "Battery"
    },
    {
      "id": 9,
      "name": "Clothing",
      "supercategory": "Clothing"
    }
  ]
}


In [6]:
coco_format['images'] = images_info


In [7]:
coco_format['images']

[{'width': 1024,
  'height': 1024,
  'file_name': 'test/0000.jpg',
  'license': 0,
  'flickr_url': None,
  'coco_url': None,
  'date_captured': '2021-01-01 13:30:03',
  'id': 0},
 {'width': 1024,
  'height': 1024,
  'file_name': 'test/0001.jpg',
  'license': 0,
  'flickr_url': None,
  'coco_url': None,
  'date_captured': '2020-12-27 17:23:39',
  'id': 1},
 {'width': 1024,
  'height': 1024,
  'file_name': 'test/0002.jpg',
  'license': 0,
  'flickr_url': None,
  'coco_url': None,
  'date_captured': '2021-04-30 11:01:01',
  'id': 2},
 {'width': 1024,
  'height': 1024,
  'file_name': 'test/0003.jpg',
  'license': 0,
  'flickr_url': None,
  'coco_url': None,
  'date_captured': '2021-03-28 23:38:43',
  'id': 3},
 {'width': 1024,
  'height': 1024,
  'file_name': 'test/0004.jpg',
  'license': 0,
  'flickr_url': None,
  'coco_url': None,
  'date_captured': '2021-01-08 00:41:15',
  'id': 4},
 {'width': 1024,
  'height': 1024,
  'file_name': 'test/0005.jpg',
  'license': 0,
  'flickr_url': None,


In [8]:
# 1. JSON 파일 불러오기
json_file_path = './work_dirs/coco_detection/test.bbox.json'  # 여기에 JSON 파일 경로를 넣어주세요
with open(json_file_path, 'r') as f:
    predictions  = json.load(f)


In [9]:
annotation_id = 1  # 어노테이션 ID는 고유해야 하므로 하나씩 증가시킴

for pred in predictions:
    # COCO 형식의 bbox는 [x, y, width, height] 형식이므로 변환할 필요 없음
    coco_annotation = {
        "id": annotation_id,
        "image_id": pred["image_id"],
        "category_id": pred["category_id"],
        "bbox": pred["bbox"],
        "score": pred.get("score", 1.0),  # score는 예측 결과이므로 optional
        "area": pred["bbox"][2] * pred["bbox"][3],  # 바운딩 박스의 넓이 계산
        "iscrowd": 0  # COCO에서는 통상 0 (일반 객체), 1은 군집 객체
    }
    coco_format['annotations'].append(coco_annotation)
    annotation_id += 1


In [10]:
output_json_path = 'tmp.json'
with open(output_json_path, 'w') as f:
    json.dump(coco_format, f, indent=4)


In [11]:
coco = COCO(output_json_path)


loading annotations into memory...
Done (t=0.91s)
creating index...
index created!


In [12]:
images_info = {img['id']: img['file_name'] for img in image_info_data['images']}


In [13]:
from collections import defaultdict

In [14]:
grouped_predictions = defaultdict(list)
for pred in predictions:
    grouped_predictions[pred['image_id']].append(pred)

In [15]:
def coco_to_voc(coco_bbox):
    x_min = coco_bbox[0]
    y_min = coco_bbox[1]
    x_max = coco_bbox[0] + coco_bbox[2]  # x_min + width
    y_max = coco_bbox[1] + coco_bbox[3]  # y_min + height
    return [x_min, y_min, x_max, y_max]

In [16]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []

img_ids = coco.getImgIds()
class_num = 10
for image_id, preds in grouped_predictions.items():
    # image_id = pred['image_id']
    
    # 예측 결과로부터 prediction string 생성
    prediction_string = ''
    for pred in preds:


        bbox = pred['bbox']

        voc_bbox = coco_to_voc(bbox)

        category_id = pred['category_id']
        score = pred['score']
        # COCO 형식의 바운딩 박스 [x, y, width, height]을 이용하여 prediction string 생성
        prediction_string += f"{category_id} {score} {voc_bbox[0]} {voc_bbox[1]} {voc_bbox[2]} {voc_bbox[3]} "


    prediction_strings.append(prediction_string.strip())  # 마지막 공백 제거
    file_names.append(images_info.get(image_id, "unknown_file"))  # 이미지 파일 이름 추가



In [17]:
import pandas as pd

In [18]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names

In [19]:
submission

,PredictionString,image_id
0,7 0.999313473701477 606.4476318359375 497.0203...,test/0000.jpg
1,5 0.9190165996551514 338.4128112792969 235.782...,test/0001.jpg
2,4 0.9736364483833313 865.44873046875 252.96948...,test/0002.jpg
3,9 0.9702824354171753 133.04898071289062 223.43...,test/0003.jpg
4,1 0.5987394452095032 183.38458251953125 289.48...,test/0004.jpg
...,...,...
4865,0 0.7186779975891113 295.98431396484375 397.63...,test/4866.jpg
4866,3 0.9274561405181885 406.77899169921875 330.27...,test/4867.jpg
4867,7 0.9895341992378235 107.798583984375 478.3505...,test/4868.jpg
4868,0 0.701500654220581 213.12911987304688 241.372...,test/4869.jpg


In [20]:
submission.to_csv('./work_dirs/coco_detection/submission_test.csv',index=False)
